# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import re


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [3]:
# Ваше решение



In [4]:
# Разнесем норматив на верхнюю и нижнюю границы
df['Нижний предел'] = 0.0
df['Верхний предел'] = 0.0

In [5]:
# Скорректируем данные:
# б/цвета = 0
# и зменим , на . в Норматив чтобы потом конвертировать во float
df.loc[df['Результат анализа'] == 'б/цвета', 'Результат анализа'] = 0
df['Норматив'] = df['Норматив'].str.replace(',','.')

In [6]:
# Из Норматив достаем нижнюю границу
low = df.loc[df['Норматив'].str.contains('в пределах'), 'Норматив'].str.extract(r'(\d{1,})-\d')
low.rename(columns={0: 'Нижний предел'}, inplace=True)
df.update(low)

In [7]:
# и верхнюю
high = df.loc[df['Норматив'].str.contains('в пределах'), 'Норматив'].str.extract(r'\d-(\d{1,})')
high.rename(columns={0: 'Верхний предел'}, inplace=True)
df.update(high)

In [8]:
# для 'не более' нижнюю принимаем за 0, а верхнюю достаем
high = df.loc[df['Норматив'].str.contains('не более'), 'Норматив'].str.extract(r'(\d{1,}\.*\d*)')
high.rename(columns={0: 'Верхний предел'}, inplace=True)
df.update(high)

In [9]:
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив,Нижний предел,Верхний предел
0,pH,единицы pH,8.4,в пределах 6-9,6,9
1,Запах,баллы,1,не более 2-3,0,2
2,Цветность,градусы,0,не более 30,0,30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,7,10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,не более 1.5,0,1.5
5,Нитриты (по NO2),мг/дм3,0.017,не более 3.3,0,3.3
6,Нитраты (по NO3),мг/дм3,24,не более 45,0,45
7,Фосфаты (P),мг/дм3,0.36,не более 3.5,0,3.5
8,Хлориды (Cl),мг/дм3,200,не более 350,0,350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,0,500


In [10]:
# конвертируем поля
df = df.astype({'Результат анализа': 'float64', 'Нижний предел': 'float64', 'Верхний предел': 'float64'})
df.dtypes

Показатель            object
Единица измерений     object
Результат анализа    float64
Норматив              object
Нижний предел        float64
Верхний предел       float64
dtype: object

In [11]:
# Сраваем показатели и записываем Статус
df.loc[(df['Результат анализа'] >= df['Нижний предел']) & (df['Результат анализа'] <= df['Верхний предел']), 'Статус'] = 'Норма'
df.loc[(df['Результат анализа'] > df['Верхний предел']), 'Статус'] = 'Превышен'

In [12]:
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив,Нижний предел,Верхний предел,Статус
0,pH,единицы pH,8.400,в пределах 6-9,6.0,9.0,Норма
1,Запах,баллы,1.000,не более 2-3,0.0,2.0,Норма
2,Цветность,градусы,0.000,не более 30,0.0,30.0,Норма
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,7.0,10.0,Норма
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,не более 1.5,0.0,1.5,Норма
5,Нитриты (по NO2),мг/дм3,0.017,не более 3.3,0.0,3.3,Норма
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,0.0,45.0,Норма
7,Фосфаты (P),мг/дм3,0.360,не более 3.5,0.0,3.5,Норма
8,Хлориды (Cl),мг/дм3,200.000,не более 350,0.0,350.0,Норма
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,0.0,500.0,Норма


In [13]:
df.set_index('Показатель')

,Единица измерений,Результат анализа,Норматив,Нижний предел,Верхний предел,Статус
Показатель,,,,,,
pH,единицы pH,8.400,в пределах 6-9,6.0,9.0,Норма
Запах,баллы,1.000,не более 2-3,0.0,2.0,Норма
Цветность,градусы,0.000,не более 30,0.0,30.0,Норма
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,7.0,10.0,Норма
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,не более 1.5,0.0,1.5,Норма
Нитриты (по NO2),мг/дм3,0.017,не более 3.3,0.0,3.3,Норма
Нитраты (по NO3),мг/дм3,24.000,не более 45,0.0,45.0,Норма
Фосфаты (P),мг/дм3,0.360,не более 3.5,0.0,3.5,Норма
Хлориды (Cl),мг/дм3,200.000,не более 350,0.0,350.0,Норма


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [14]:
# Ваше решение
box = ['апельсин' for x in range(5)] + ['яблоко' for x in range(4)]
box

['апельсин',
 'апельсин',
 'апельсин',
 'апельсин',
 'апельсин',
 'яблоко',
 'яблоко',
 'яблоко',
 'яблоко']

In [15]:
%%time
N = 1000000
cases = [np.random.choice(box, 3) for x in range(N)]

CPU times: user 37.4 s, sys: 2.68 s, total: 40.1 s
Wall time: 38.2 s


In [16]:
df_fruits = pd.DataFrame(cases, columns=['first', 'second', 'third'])
display(df_fruits)

,first,second,third
0,апельсин,апельсин,апельсин
1,апельсин,яблоко,апельсин
2,апельсин,апельсин,апельсин
3,апельсин,яблоко,апельсин
4,апельсин,апельсин,яблоко
...,...,...,...
999995,апельсин,яблоко,яблоко
999996,апельсин,апельсин,яблоко
999997,апельсин,апельсин,яблоко
999998,яблоко,яблоко,яблоко


In [17]:
result = len(df_fruits.query('first == "апельсин" & second == "апельсин" & third == "апельсин"'))

In [18]:
display("Три фрукта – апельсины: {} раз".format(result))
display("Вероятность: {}".format(result/df_fruits.shape[0]))

'Три фрукта – апельсины: 171238 раз'

'Вероятность: 0.171238'

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [19]:
# Ваше решение
box = ['стандартная' for x in range(7)] + ['нестандартная' for x in range(3)]
box

['стандартная',
 'стандартная',
 'стандартная',
 'стандартная',
 'стандартная',
 'стандартная',
 'стандартная',
 'нестандартная',
 'нестандартная',
 'нестандартная']

In [20]:
%%time
N = 1000000
cases = [np.random.choice(box, 2) for x in range(N)]

CPU times: user 38.3 s, sys: 3.46 s, total: 41.7 s
Wall time: 39.1 s


In [21]:
df_tool = pd.DataFrame(cases, columns=['first', 'second'])

In [22]:
result = len(df_tool.query('first == "нестандартная" & second == "стандартная"'))

In [23]:
display("нестандартная, стандартная: {} раз".format(result))
display("Вероятность: {}".format(result/df_tool.shape[0]))

'нестандартная, стандартная: 209676 раз'

'Вероятность: 0.209676'